# Beating the Market with AI

### Initial Project Setup

In [ ]:
# Initial library and module imports
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
import matplotlib.pyplot as plt

# Example imports for models and algorithms

# from sklearn import svm
# from pandas.tseries.offsets import DateOffset
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report

# Disable false positive error message
# pd.options.mode.chained_assignment = None

In [ ]:
# Initial file uploads for Google Colab
from google.colab import files
uploaded = files.upload()

### Data Import & Preparation

In [ ]:
# Include both Google Colab and non-colab import options
# Ex.
# Read csv into dataframe - Colab
# applicant_data_df = pd.read_csv(Path("./applicants_data.csv"))

# Read csv into dataframe - Non-Colab
# applicant_data_df = pd.read_csv(Path("./Resources/applicants_data.csv"))

### Stock A Algorithm - Conservative

### Stock B Algorithm - Aggressive

### Stock A Models

#### Model A

#### Model B

#### Model C?

### Stock B Models

#### Model A

#### Model B

#### Model C?

### Analysis & Comparison

In [ ]:
# Compare Algo A & B as well as all models to identify the best algo and model combos